## MIIA-4203 MODELOS AVANZADOS PARA ANÁLISIS DE DATOS II


# Introducción a Sistemas de Recomendación

## Actividad 10


### Profesor: Camilo Franco (c.franco31@uniandes.edu.co)

## Actividad en grupos
### Nombres: Lupe Pizón, Camilo Angarita, Juan Andrés Cabal

**Instrucciones:** Por favor escriba los nombres de los integrantes de su grupo. Esta actividad debe ser entregada a más tardar dentro de 8 días, con la respuesta para los ejercicios y preguntas en cada numeral.

En este cuadernos estudiaremos una primera introducción a los sistemas de recomendación, examinando un modelo de recomendación en base a la popularidad, sin ningún grado de personalización. En futuras actividades exploraremos otros sistemas basados en contenido o en filtrado colaborativo, los cuales toman en cuenta las preferencias de los usuarios. 

Los sistemas de recomendación están entre las aplicaciones más populares de la ciencia de datos actualmente. El marco que ofrecen estos sistemas permite representar modelos de decisión junto con métodos estadístico/computacionales para **aprender y estimar** las preferencias de los usuarios. 


## Introducción

En esta introducción vamos a construir un modelo simple basado en popularidad. Aprenderemos nociones que nos permitirán construir modelos más complejos de recomendación. 

En este cuaderno vamos a trabajar con una base de datos de películas IMDB (https://www.imdb.com/) 

En términos generales, los sistemas de recomendación pueden clasificarse en 3 tipos:

- **Recomendadores simples:** genera recomendaciones para cada usuario según la popularidad del ítem o producto. Por ejemplo, proponer películas o un género en base a su popularidad o su crítica, donde una buena crítica aproxima una mayor probabilidad de que la película sea apreciada por una audiencia "promedio".
- **Recomendadores basados en contenido:** sugiere elementos similares en función de un elemento en particular, como por ejemplo el género, la sinopsis o el reparto (actores, etc). De esta manera, si el usuario tiene unas preferencias específicas sobre un ítem/artículo en particular, también podría tener preferencia por un ítem *similar*.
- **Recomendadores por filtrado colaborativo:** estos sistemas generan recomendaciones a partir de similitudes entre usuarios, prediciendo la calificación o preferencia de un usuario sobre un ítem, o un grupo de atributos de dicho ítem, según las calificaciones y preferencias observadas de otros usuarios. 

Por último se puede considerar también un cuarto tipo de recomendadores, que son los de tipo **Híbrido**. Estos hacen referencia al uso simultaneo de las distintas metodologias antes señaladas y de múltiples fuentes de información o tecnologías específicas, como pueden ser las redes sociales o fuentes de información geográfica, demográfica, etc.

Para profundizar en los sistemas de recomendación se pueden consultar las siguientes fuentes:
- Ch. Aggarwal. Recommender Systems. Springer, 2016.
- J. Leskovec, A. Rajaraman, J. Ullman. Mining of Massive Datasets. Cambridge University Press (http://www.mmds.org/)


## 1. Sistema de recomendación simple

Los recomendadores simples generan recomendaciones basadas en su popularidad, de acuerdo con una *preferencia promedio*. 

La pregunta es,

**¿cómo medimos la popularidad de un producto/ítem?**

Una opción es la de elegir una métrica determinada para puntuar las películas, y en base a esa métrica generar un valor de preferencia por el ítem. De esta manera es posible consturir un *orden lineal* o un *ranking estricto* con las mejores películas de la base de datos IMDB.

Pasos a seguir:
- Elegir la métrica con la que se va a calificar cada película. Este punto también se puede entender como un modelo para estimar las preferencias por una película.
- Calcular el valor de preferencia sobre cada película.
- Ordenar las películas según su valor o puntuación y obtener una lista con las mejores k peliculas.

Manos a la obra, primero carguemos el conjunto de datos como un data frame en la bilbioteca Pandas:

In [1]:
# Importamos la biblioteca Pandas
import pandas as pd

# Cargamos los datos de peliculas de la base de datos IMDB
metadata = pd.read_csv('movies_metadata.csv', low_memory=False)

print(metadata.shape)
      
list(metadata)

(45466, 24)


['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [2]:
# Así se ven los datos
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [3]:
metadata[['original_title', 'popularity']][1:10]

,original_title,popularity
1,Jumanji,17.015539
2,Grumpier Old Men,11.7129
3,Waiting to Exhale,3.859495
4,Father of the Bride Part II,8.387519
5,Heat,17.924927
6,Sabrina,6.677277
7,Tom and Huck,2.561161
8,Sudden Death,5.23158
9,GoldenEye,14.686036


### Rating vs. Popularidad

Con esta información vamos a construir un sistema que recomiende películas basados directamente en su *rating*, o **voto promedio**. Sin embargo, es cierto que hay otro atributo relevante como lo es el de su **popularidad**. Por ejemplo, si un solo usuario califica con 10 una pelicula, pero 5000 usuarios le dan en promedio un 8.3, solo en base al rating la primera película sería considerada mejor que la segunda.


Bajo una perspectiva de *los grandes números*, la valoración o voto promedio de una película tiende a converger hacia un valor que refleje la calidad de la película a medida que el número de votos aumenta. Con poca información, o pocos votos, es más difícil estimar la calidad promedio de la película.  


Entonces, 

**¿qué tipo de métrica nos sirve para evaluar apropiadamente la preferencia por una película?** 

Tengamos en cuenta que en base a esta métrica se tendrán mejores o peores recomendaciones!




### Ejercicio 1.1

Examine los datos y proponga una métrica que permita estimar la preferencia promedio por una película.

In [24]:
metadata = metadata.drop(index=(metadata.loc[metadata['popularity'] == 'Beware Of Frost Bites'].index))
metadata['popularity'] = metadata['popularity'].astype(float)
metadata[['original_title', 'homepage', 'vote_count', 'vote_average', 'popularity']].describe()

,vote_count,vote_average,popularity
count,45460.000000,45460.000000,45460.000000
mean,109.897338,5.618207,2.921478
std,491.310374,1.924216,6.005414
min,0.000000,0.000000,0.000000
25%,3.000000,5.000000,0.385948
50%,10.000000,6.000000,1.127685
75%,34.000000,6.800000,3.678902
max,14075.000000,10.000000,547.488298


Al examinar el número de votos, el promedio de votaciones y la popularidad de las películas es posible encontrar que, en promedio, el número de votos es igual a 109 mientras que el voto promedio es igual a 5.61 en una escala de 10. De igual forma, de las estadísticas descriptivas es posible observar que el promedio de popularidad de las películas es igual a 2.92, considerablemente un valor menor al rating promedio de las películas. 

Del análisis inicial es posible ver, de igual forma, que la variable popularidad, aunque puede servir como insumo para estimar la preferencia de una película, no se encuentra normalizada, siendo el valor máximo igual a 547.48 que corresponde a la película de los Minions, la cual tiene un rating promedio igual a 6.4. 

Ahora, dado que lo que se busca es proponer una métrica (diferente a la de rating_ponderado) que permita estimar la preferencia promedio por una película se hara uso de una calificación ponderada que tenga en cuenta tanto el ranting que recibe una película por su número de votos, así como el voto promedio global de las películas por el número de votos que recibe cada una de estas. Matemáticamente, se representa de la siguiente manera:

$$
\mu_i  = \left( \frac{v_i}{v_i+m_i} \right) R_i + \left(\frac{m_i}{v_i+m_i} \right) C 
$$

donde $v_i$ es el número de votos para la $i$-ésima película, $m_i$ es el mínimo de votos requerido para ser incluido en el cuadro, $R$ es el rating promedio de la pelicula y $C$ es el voto promedio global de todas las películas. 

In [26]:
def IMDB_rating(x, C):
    """
    Input:
    x: datos de rating y votacion de las peliculas
    m: minimo numero de votos
    C: promedio global
    Output:
    rating ponderado
    """
    m = x['vote_average'].max()
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

### Ponderación entre rating y popularidad

Una opción puede consistir en ponderar el rating que recibe una película por su número de votos. De esta manera se pueden prevenir situaciones donde una película con un voto de 10 obtenga una mejor calificación que una película con 1000 votos con un promedio de 9.5.  

De esta manera, el voto promedio ponderado $\mu_i$, de la $i$-ésima película lo definimos como:

$$
\mu_i  = \left( \frac{v_i}{v_{max}} \right) R_i 
$$

donde $v_i$ es el número de votos para la $i$-ésima película, $v_{max}$ es el máximo número de votos que recibe la película más popular, y $R$ es el rating promedio de la pelicula.


En la base de datos tenemos los valores de $v_i$ (vote_count) y $R$ (vote_average) para cada pelicula. Además podemos agregar un filtro para que considere las películas con un minimo número de votos $v_{min}$.


Nos quedamos con las columnas necesarias para calcular el rating ponderado más el link de la pagina (en caso de que esté disponible):

In [27]:
pelisR = metadata[['original_title', 'homepage', 'vote_count', 'vote_average']]
pelisR.shape

(45465, 4)

In [28]:
# Calificacion de peliculas basada en la ponderacion de su rating 
def rating_ponderado(x):
    """
    Input:
    x: datos de rating y votacion de las peliculas
    m: minimo numero de votos
    Output:
    rating ponderado
    """
    m = x['vote_average'].max()
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) 

Definimos una nueva respuesta 'score' y calculamos su valor con la funcion rating_ponderado().

A continuación ordenamos las películas basados en el score y ofrecemos el top 15 de películas preferidas:

In [29]:
import warnings
warnings.filterwarnings("ignore", category=Warning)

# si tiene al menos vmin votos, incluimos la pelicula
vmin = 1
pelisRF = pelisR.copy().loc[pelisR['vote_count'] >= vmin]
pelisRF['score'] = rating_ponderado(pelisRF)

# ordenamos las peliculas de acuerdo con el score
pelisRF = pelisRF.sort_values('score', ascending=False)

# Mostramos el top15 de las peliculas
pelisRF[['original_title', 'homepage', 'vote_count', 'vote_average', 'score']].head(15)

,original_title,homepage,vote_count,vote_average,score
10309,Dilwale Dulhania Le Jayenge,NaN,661.0,9.1,8.964382
314,The Shawshank Redemption,NaN,8358.0,8.5,8.489842
834,The Godfather,http://www.thegodfather.com/,6024.0,8.5,8.485913
40251,君の名は。,https://www.funimationfilms.com/movie/yourname/,1030.0,8.5,8.418269
39085,Planet Earth,NaN,176.0,8.8,8.326882
12481,The Dark Knight,http://thedarkknight.warnerbros.com/dvdsite/,12269.0,8.3,8.293240
2843,Fight Club,http://www.foxmovies.com/movies/fight-club,9678.0,8.3,8.291433
292,Pulp Fiction,NaN,8670.0,8.3,8.290438
522,Schindler's List,http://www.schindlerslist.com/,4436.0,8.3,8.281332
23673,Whiplash,http://sonyclassics.com/whiplash/,4376.0,8.3,8.281076


### Examinemos el efecto de cambiar el mínimo numero de votos a considerar:

In [30]:
# si tiene al menos mil votos incluimos la pelicula
vmin = 1000
pelisRF = pelisR.copy().loc[pelisR['vote_count'] >= vmin]
pelisRF['score'] = rating_ponderado(pelisRF)

# ordenamos las peliculas de acuerdo con el score
pelisRF = pelisRF.sort_values('score', ascending=False)

# Mostramos el top15 de las peliculas
pelisRF[['original_title', 'homepage', 'vote_count', 'vote_average', 'score']].head(15)

,original_title,homepage,vote_count,vote_average,score
314,The Shawshank Redemption,NaN,8358.0,8.5,8.491364
834,The Godfather,http://www.thegodfather.com/,6024.0,8.5,8.488023
40251,君の名は。,https://www.funimationfilms.com/movie/yourname/,1030.0,8.5,8.430429
12481,The Dark Knight,http://thedarkknight.warnerbros.com/dvdsite/,12269.0,8.3,8.294254
2843,Fight Club,http://www.foxmovies.com/movies/fight-club,9678.0,8.3,8.292717
292,Pulp Fiction,NaN,8670.0,8.3,8.291871
522,Schindler's List,http://www.schindlerslist.com/,4436.0,8.3,8.284126
23673,Whiplash,http://sonyclassics.com/whiplash/,4376.0,8.3,8.283909
5481,千と千尋の神隠し,http://movies.disney.com/spirited-away,3968.0,8.3,8.282258
2211,La vita è bella,NaN,3643.0,8.3,8.280679


Veamos en qué consiste esa pelicula en japonés:

In [31]:
import webbrowser

webbrowser.open(pelisRF['homepage'][40251]) 

True

### Ejercicio 1.2

Examine cómo varían los resultados si tomamos directamente la variable de popularidad.

Con el fin de hacer los resultados anteriores comparables, se examinará como varían los resultados si se toma directamente la variable de popularidad sobre aquellas películas que tengan al menos mil votos incluidos. En este caso, se observa un resultado diferente al que se obtiene al utilizar la métrica *ranting_ponderado*, siendo Minioms la primera película de la lista, seguido por Wonder Woman. De hecho, al examinar el top 15 de películas que se encuentran cuando se utiliza la variable popularidad, ninguna de estás hacen parte del top 15 que se encuentra cuando se utiliza el *ranting_ponderado*.

Este resultado muestran, en últimas, que la métrica que se utilice para evaluar apropiadamente la preferencia de una película puede mejorar o empeorar el resultado de la recomendación. En ese sentido, es importante entender la estructura de la base de datos y tener bajo consideración como se distribuyen los datos para construir un indicador apropiado que permita hacer una recomendación valida. 

In [34]:
pelisR = metadata[['original_title', 'homepage', 'vote_count', 'vote_average', 'popularity']]

# si tiene al menos mil votos incluimos la pelicula
vmin = 1000
pelisPF = pelisR.copy().loc[pelisR['vote_count'] >= vmin]

# ordenamos las peliculas de acuerdo con la popularidad
pelisPF = pelisPF.sort_values('popularity', ascending=False)

# Mostramos el top15 de las peliculas
pelisPF[['original_title', 'homepage', 'vote_count', 'vote_average', 'popularity']].head(15)

,original_title,homepage,vote_count,vote_average,popularity
30700,Minions,http://www.minionsmovie.com/,4729.0,6.4,547.488298
33356,Wonder Woman,http://www.warnerbros.com/wonder-woman,5025.0,7.2,294.337037
42222,Beauty and the Beast,http://movies.disney.com/beauty-and-the-beast-...,5530.0,6.8,287.253654
43644,Baby Driver,NaN,2083.0,7.2,228.032744
24455,Big Hero 6,http://movies.disney.com/big-hero-6,6289.0,7.8,213.849907
26564,Deadpool,http://www.foxmovies.com/movies/deadpool,11444.0,7.4,187.860492
26566,Guardians of the Galaxy Vol. 2,http://marvel.com/movies/movie/221/guardians_o...,4858.0,7.6,185.330992
14551,Avatar,http://www.avatarmovie.com/,12114.0,7.2,185.070892
24351,John Wick,http://www.johnwickthemovie.com/,5499.0,7.0,183.870374
23675,Gone Girl,http://www.gonegirlmovie.com/,6023.0,7.9,154.801009


### Recomendación de peliculas mas populares por genero

In [41]:

metadata['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genres, Length: 45465, dtype: object

In [43]:
import numpy as np
from ast import literal_eval

# trabajamos la informacion por generos
metadata['genres'] = metadata['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

# añadimos la variable del año
metadata['year'] = pd.to_datetime(metadata['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995


Primero nos quedamos con todos los generos:

In [46]:
metadata.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)

0        Animation
0           Comedy
0           Family
1        Adventure
1          Fantasy
           ...    
45461       Family
45462        Drama
45463       Action
45463        Drama
45463     Thriller
Length: 91102, dtype: object

In [47]:
generos = metadata.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
generos.name = 'genre'
gen_md = metadata.drop('genres', axis=1).join(generos)

gen_md.head(3)

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Animation
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Comedy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Family


Construimos una funcion para un género particular y que tome en cuenta peliculas con un número vmin de votos:

In [48]:
def rec_gen(genero, vmin):
    df = gen_md[gen_md['genre'] == genero]
    v = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    R = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    m = df['vote_average'].max()
    
    pelisG = df[(df['vote_count'] >= vmin) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'overview', 'homepage']]
    pelisG['vote_count'] = pelisG['vote_count'].astype('int')
    pelisG['vote_average'] = pelisG['vote_average'].astype('int')
    
    pelisG['wr'] = v/m * R
    pelisG = pelisG.sort_values('wr', ascending=False).head(250)
    
    return pelisG

Veamos el Top-15 de recomendaciones en Ciencia Ficción:

In [49]:
k = 15
scifi = rec_gen('Science Fiction', 1000)
scifi.head(15)

,title,year,vote_count,vote_average,popularity,overview,homepage,wr
15480,Inception,2010,14075,8,29.108149,"Cobb, a skilled thief who commits corporate es...",http://inceptionmovie.warnerbros.com/,11260.0
22879,Interstellar,2014,11187,8,32.213481,Interstellar chronicles the adventures of a gr...,http://www.interstellarmovie.net/,8949.6
14551,Avatar,2009,12114,7,185.070892,"In the 22nd century, a paraplegic Marine is di...",http://www.avatarmovie.com/,8479.8
17818,The Avengers,2012,12000,7,89.887648,When an unexpected enemy emerges and threatens...,http://marvel.com/avengers_movie/,8400.0
23753,Guardians of the Galaxy,2014,10014,7,53.291601,"Light years from Earth, 26 years after being a...",http://marvel.com/guardians,7009.8
26553,Mad Max: Fury Road,2015,9629,7,29.361780,An apocalyptic story set in the furthest reach...,http://www.madmaxmovie.com/,6740.3
2458,The Matrix,1999,9079,7,33.366332,"Set in the 22nd century, The Matrix tells the ...",http://www.warnerbros.com/matrix,6355.3
12588,Iron Man,2008,8951,7,22.073099,"After being held captive in an Afghan cave, bi...",http://www.ironmanmovie.com/,6265.7
18244,The Hunger Games,2012,9634,6,20.031667,Every year in the ruins of what was once North...,http://www.thehungergames.movie/,5780.4
26555,Star Wars: The Force Awakens,2015,7993,7,31.626013,Thirty years after defeating the Galactic Empi...,http://www.starwars.com/films/star-wars-episod...,5595.1


Inception e Interstellar aparecen en las dos primeras posiciones. Personalmente me gusta más Interstellar (si quieres ver un agujero negro, esta película es lo mejor que podrás conseguir), pero reconozco que Inception tiene mucho nivel. Podemos seguir refinando este tipo de recomendaciones prestando atención a los distintos atributos que tenemos disponibles sobre las películas. 


### Ejercicio 1.3

Elija el género de su interés y aplique la métrica propuesta en el Ejercicio 1.1

## Genero: Aventura

In [68]:
C = df['vote_average'].mean()
df = gen_md[gen_md['genre'] == 'Adventure']

# si tiene al menos mil votos incluimos la pelicula
pelisG1 = df[(df['vote_count'] >= vmin) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'overview', 'homepage', 'genre']]

# ordenamos las peliculas de acuerdo con la popularidad
pelisG1['score'] = IMDB_rating(pelisG1, C)
pelisG1 = pelisG1.sort_values('score', ascending=False)

# Mostramos el top15 de las peliculas
pelisG1.head(15)

,title,year,vote_count,vote_average,popularity,overview,homepage,genre,score
5481,Spirited Away,2001,3968.0,8.3,41.048867,A ten year old girl who wanders away from her ...,http://movies.disney.com/spirited-away,Adventure,8.294544
1154,The Empire Strikes Back,1980,5998.0,8.2,19.470959,"The epic saga continues as Luke Skywalker, in ...",http://www.starwars.com/films/star-wars-episod...,Adventure,8.196526
9698,Howl's Moving Castle,2004,2049.0,8.2,16.136048,"When Sophie, a shy young woman, is cursed with...",NaN,Adventure,8.189858
2884,Princess Mononoke,1997,2041.0,8.2,17.166725,"Ashitaka, a prince of the disappearing Ainu tr...",NaN,Adventure,8.189818
15480,Inception,2010,14075.0,8.1,29.108149,"Cobb, a skilled thief who commits corporate es...",http://inceptionmovie.warnerbros.com/,Adventure,8.098577
22879,Interstellar,2014,11187.0,8.1,32.213481,Interstellar chronicles the adventures of a gr...,http://www.interstellarmovie.net/,Adventure,8.098210
7000,The Lord of the Rings: The Return of the King,2003,8226.0,8.1,29.324358,Aragorn is revealed as the heir to the ancient...,http://www.lordoftherings.net,Adventure,8.097567
256,Star Wars,1977,6778.0,8.1,42.149697,Princess Leia is captured and held hostage by ...,http://www.starwars.com/films/star-wars-episod...,Adventure,8.097048
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892.0,8.0,32.070725,"Young hobbit Frodo Baggins, after inheriting a...",http://www.lordoftherings.net/,Adventure,7.997842
5814,The Lord of the Rings: The Two Towers,2002,7641.0,8.0,29.423537,Frodo and Sam are trekking to Mordor to destro...,http://www.lordoftherings.net/,Adventure,7.997489


Spirited Away y The Empire Strikes Back aparecen en las dos primeras posiciones cuando se miran las películas cuyo genero es de aventura. Ambas, ciertamente, han sido reconocidas como grandes películas. 

## Genero: Familiar

In [75]:
C = df['vote_average'].mean()
df = gen_md[gen_md['genre'] == 'Family']

# si tiene al menos mil votos incluimos la pelicula
pelisG1 = df[(df['vote_count'] >= vmin) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'overview', 'homepage', 'genre']]

# ordenamos las peliculas de acuerdo con la popularidad
pelisG1['score'] = IMDB_rating(pelisG1, C)
pelisG1 = pelisG1.sort_values('score', ascending=False)

# Mostramos el top15 de las peliculas
pelisG1.head(15)

,title,year,vote_count,vote_average,popularity,overview,homepage,genre,score
5481,Spirited Away,2001,3968.0,8.3,41.048867,A ten year old girl who wanders away from her ...,http://movies.disney.com/spirited-away,Family,8.294684
1225,Back to the Future,1985,6239.0,8.0,25.778509,Eighties teenager Marty McFly is accidentally ...,http://www.backtothefuture.com/movies/backtoth...,Family,7.997015
359,The Lion King,1994,5520.0,8.0,21.605761,A young lion cub named Simba can't wait to be ...,http://movies.disney.com/the-lion-king,Family,7.996627
5833,My Neighbor Totoro,1988,1730.0,8.0,13.507299,Two sisters move to the country with their fat...,NaN,Family,7.989273
926,It's a Wonderful Life,1946,1103.0,8.0,15.031588,George Bailey has spent his entire life giving...,NaN,Family,7.983221
30315,Inside Out,2015,6737.0,7.9,23.985587,"Growing up can be a bumpy road, and it's no ex...",http://movies.disney.com/inside-out,Family,7.897359
17437,Harry Potter and the Deathly Hallows: Part 2,2011,6141.0,7.9,24.990737,"Harry, Ron and Hermione continue their quest t...",http://www.harrypotter.com,Family,7.897103
13724,Up,2009,7048.0,7.8,19.330884,Carl Fredricksen spent his entire life dreamin...,http://disney.go.com/disneypictures/up/,Family,7.797593
12704,WALL·E,2008,6439.0,7.8,16.088366,WALL·E is the last robot left on an Earth that...,http://disney.go.com/disneypictures/wall-e/,Family,7.797365
24455,Big Hero 6,2014,6289.0,7.8,213.849907,The special bond that develops between plus-si...,http://movies.disney.com/big-hero-6,Family,7.797303


Al seleccionar un segundo genero: "Familiar" Spirited Away y The Empire Strikes Back aparecen nuevamente en las dos primeras posiciones. Lo que refuerza la popularidad de las dos peliculas